# 爬內文及推文數

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from multiprocessing import Pool
from time import sleep

In [92]:
all_res = pd.read_csv("info_all.csv")
all_res_title = all_res["Title"]
all_res_title

0                        [問卦] 八卦板取年號該取啥
1            [新聞] 高雄簽13億無法律效力MOU 陳吉仲：農委
2              [問卦] 有沒有雨衣只是把雨水集中到鞋子裡的八卦
3            [新聞] 紐航將開通直飛首爾航班 機上Wifi將免費
4                    [問卦] 有沒有賣國旗的外國人的八卦
5              [新聞] 去中國工作待遇好？蔡依橙爆上海、台北起
6                    [問卦] 所以為什麼要隱瞞有蜥蜴人?
7                [問卦] 平常如果都反串 愚人節時該怎麼辦?
8               [問卦] 為什麼華人YT訂閱數難以超越外國人?
9               [臉書] 拷秋勤 Kou Chou Ching
10        Re: [新聞] 綠島媽貼文公審女醫 台大醫：2公分傷口哪
11                     [問卦] 14台到底是播什麼的？
12           Re: [問卦] 如何正名秘書才是真正的尊重專業!?
13             [新聞] 兒子拿刀逼母要錢 母哭：留1千給我看病
14                  [問卦] 淘寶是不是一堆賣家支持台獨?
15                 [新聞] 微軟發現華為筆記型電腦存在後門
16                        [問卦] 外國女孩在發酒瘋
17              [新聞] 新北旅宿涉不法 第二次查到就勒令停業
18                       [問卦] 有沒有雷蛙的八卦？
19                   Re: [問卦] 中日哪個年號最威呀
20                   [問卦] 買二手手機的是不是腦子有病
21                      [問卦] 如何搭訕赤燭系女孩?
22                       [問卦] 有沒有18台的八卦
23              [問卦] 只會中文或只會英文那個在台灣比較吃香
24             [新聞] 北市明星高中現狼師性侵學生 教育局證實


In [3]:
def getPageContent(URL):
    '''取得頁面原始碼'''
    
    res = requests.get(url = URL, cookies = {'over18':'1'})
    soup = BeautifulSoup(res.text)
    return soup

In [4]:
def getPageLinks(start_page, page_want_to_crawl):
    '''要爬的起始頁數 & 總頁數'''
    
    links = []
    for i in range(start_page, page_want_to_crawl, -1):
        link = 'https://www.ptt.cc/bbs/Gossiping/index' + str(i) + '.html'
        links.append(link)
        
    return links

In [5]:
def getEachRow(URL):
    '''文章列表的所有文章，包含時間、推文總和、標題、作者、連結'''
    
    soup = getPageContent(URL)
    content = soup.find_all('div', {'class': 'r-ent'})
    
    link = []
    title = []
    date = []
    author = []
    push = []
    
    for i in content:
        url = i.find('a')
        if url:
            link.append(url['href']),
            title.append(url.text)
            date.append(i.find('div', 'date').text)
            author.append(i.find('div', 'author').text)
            push.append(i.find('div', 'nrec').text)
            
    info = pd.DataFrame({'Date': date, 'Push': push, 'Title': title, 'Author': author, 'Link': link})
    return info

In [21]:
def getArticles(URL):
    '''單篇文章的資訊，包含時間、回覆總數、作者、標題、內容、文章網址'''

    #內文
    res = requests.get(url = 'https://www.ptt.cc' + URL, cookies = {'over18':'1'})
    soup = BeautifulSoup(res.text, 'html.parser')
    soup2 = BeautifulSoup(res.text, 'html.parser')
    
    try:
        main_content = soup2.find_all('div', 'bbs-screen bbs-content')
        information = soup2.find_all('div', ['richcontent', 'article-metaline', 'article-metaline-right', 'push'])
        edit = soup2.find_all('span', 'f2')
#         刪除不要的標籤
        for i in information:
            i = i.extract()
        for i in edit:
            i = i.extract()

        content = []
        for i in main_content:
            words = i.text.strip()
        #print(words)
        content.append(words)
    except:
        content = ['404 not found']
        
    # 總回覆數計算
    com = soup.find_all('div', 'push')
    count = 0

    for elem in com:
        if elem.find('span', 'f1 hl push-tag'):
            count += 1
            
        if elem.find('span', 'hl push-tag'):
            count += 1
    #print(count)
    num = [count]

    
    info = pd.DataFrame({'Push_Count' : num , 'Content':content})
    #try:
        #info['Push'] = pd.to_datetime(info['Push'])
    #except:
        #pass
    
    return info

In [22]:
test_0 = all_res["Link"]
test_0_0 = test_0[0]
test_0_0
getArticles(test_0_0)

,Push_Count,Content
0,17,各位八卦鄉民們晚安\n\n年號通常有皇帝對於未來期許的意味\n\n如果八卦板也來年號紀元\n...


In [40]:
test = all_res["Link"]
test_0_100 = test[0:100]
#test_0_100

In [24]:
content_0_100 = pd.DataFrame(columns=['Push_Count','Content'])
count = 0

if __name__ == '__main__':
    for link in test_0_100:
        df = getArticles(link)
        content_0_100 = pd.concat([content_0_100, df], ignore_index=True, sort=False)
        count += 1
        
        if count % 100 == 0:
            sleep(5)
content_0_100.head()

,Push_Count,Content
0,17,各位八卦鄉民們晚安\n\n年號通常有皇帝對於未來期許的意味\n\n如果八卦板也來年號紀元\n...
1,27,1.媒體來源:\n\n\n自由\n2.完整新聞標題:\n\n高雄簽13億無法律效力MOU 陳...
2,27,台灣人最愛騎機車，\n\n一遇到下雨就只能穿雨衣，\n\n但雨衣的設計根本就沒有辦法騎機車，...
3,10,大紀元\n\n\n紐航將開通直飛首爾航班 機上Wifi將免費\n\n\n【大紀元2019年0...
4,8,之前有個自稱聾啞的外國人拿著國旗給路人\n路人如果伸手拿了 就會拿出張紙寫一支國旗100塊...


# 先爬0到9999共一萬筆資料

In [49]:
link_all = all_res["Link"]
link_0_10000 = link_all[0:10000]
len(link_0_10000)
link_0_10000.tail()

9995    /bbs/Gossiping/M.1554542842.A.092.html
9996    /bbs/Gossiping/M.1554542879.A.110.html
9997    /bbs/Gossiping/M.1554542897.A.691.html
9998    /bbs/Gossiping/M.1554542914.A.64A.html
9999    /bbs/Gossiping/M.1554542957.A.625.html
Name: Link, dtype: object

In [34]:
content_0_10000 = pd.DataFrame(columns=['Push_Count','Content'])
count = 0

if __name__ == '__main__':
    for link in link_0_10000:
        df = getArticles(link)
        content_0_10000 = pd.concat([content_0_10000, df], ignore_index=True, sort=False)
        count += 1
        
        if count % 100 == 0:
            sleep(5)
            
content_0_10000.head()

,Push_Count,Content
0,17,各位八卦鄉民們晚安\n\n年號通常有皇帝對於未來期許的意味\n\n如果八卦板也來年號紀元\n...
1,27,1.媒體來源:\n\n\n自由\n2.完整新聞標題:\n\n高雄簽13億無法律效力MOU 陳...
2,27,台灣人最愛騎機車，\n\n一遇到下雨就只能穿雨衣，\n\n但雨衣的設計根本就沒有辦法騎機車，...
3,10,大紀元\n\n\n紐航將開通直飛首爾航班 機上Wifi將免費\n\n\n【大紀元2019年0...
4,8,之前有個自稱聾啞的外國人拿著國旗給路人\n路人如果伸手拿了 就會拿出張紙寫一支國旗100塊...


In [41]:
content_0_10000.to_csv('content_0_10000.csv', columns=['Push_Count', 'Content'])

# 爬10000到20000

In [50]:
link_all = all_res["Link"]
link_10000_20000 = link_all[10000:20000]
len(link_10000_20000)
link_10000_20000.tail()

19995    /bbs/Gossiping/M.1554934972.A.EB5.html
19996    /bbs/Gossiping/M.1554935151.A.697.html
19997    /bbs/Gossiping/M.1554935193.A.045.html
19998    /bbs/Gossiping/M.1554935496.A.C24.html
19999    /bbs/Gossiping/M.1554935601.A.C5C.html
Name: Link, dtype: object

In [52]:
content_10000_20000 = pd.DataFrame(columns=['Push_Count','Content'])
count = 0

if __name__ == '__main__':
    for link in link_10000_20000:
        df = getArticles(link)
        content_10000_20000 = pd.concat([content_10000_20000, df], ignore_index=True, sort=False)
        count += 1
        
        if count % 100 == 0:
            sleep(5)
            
content_10000_20000.head()

,Push_Count,Content
0,49,有些歌很好聽，卻不是每個人唱都能駕馭起來\n\n甚至只有本人才唱得出那種味道，聲音搭配恰到好...
1,24,: 不認東區沒落？柯文哲：是台灣經濟沒成長\n: 2019/04/06 11:27:27\n...
2,23,不知道為什麼？\n本季兩個動畫名字都有OO\n一個人OO的小日子\n滿腦子都是OO的我沒辦法...
3,14,覺得這種話不要胡說!\n\n\n\n40處男 才是怪咖!\n\n40處男 才是怪咖!!!\n...
4,16,--\n ▋ \n ╭──╮ ▎ \n ...


In [53]:
content_10000_20000.to_csv('content_10000_20000.csv', columns=['Push_Count', 'Content'])

In [60]:
#測試合併
merge_df = pd.concat([content_0_10000 , content_10000_20000] , axis = 0 , ignore_index = True)
len(merge_df)

20000

# 爬20000到30000

In [62]:
link_all = all_res["Link"]
link_20000_30000 = link_all[20000:30000]
len(link_20000_30000)
link_20000_30000.tail()

29995    /bbs/Gossiping/M.1555335290.A.2B0.html
29996    /bbs/Gossiping/M.1555335294.A.C7D.html
29997    /bbs/Gossiping/M.1555335306.A.E2B.html
29998    /bbs/Gossiping/M.1555335324.A.76E.html
29999    /bbs/Gossiping/M.1555335324.A.9B1.html
Name: Link, dtype: object

In [72]:
content_20000_30000 = pd.DataFrame(columns=['Push_Count','Content'])
count = 0

if __name__ == '__main__':
    for link in link_20000_30000:
        df = getArticles(link)
        content_20000_30000 = pd.concat([content_20000_30000, df], ignore_index=True, sort=False)
        count += 1
        
        if count % 100 == 0:
            sleep(5)
            
content_20000_30000.head()

,Push_Count,Content
0,339,中時 林宏聰\n\n選總統太敏感？韓國瑜哈佛演講規模一變再變\n\n高雄市長韓國瑜赴美，將...
1,12,: 目前的蔡賴讓我不禁聯想到之前高雄縣市合併\n: 高雄縣市合併後的小巨人（賴）陳菊（蔡）之...
2,14,: 大家有沒有覺得 這兩年的台劇進步很多\n: 雖然公視一直都拍良心劇\n: 但問題是 很多...
3,3,本魯中西醫都有朋友\n沒有哪個有用、哪個沒用\n每種治療方式都會進步\n硬要說的話，中西醫治...
4,3,: 如題\n: 最近看到獅子王的寫實版預告影片\n: 有一幕是原本的獅子王被刀疤設計害死，辛...


In [73]:
content_20000_30000.to_csv('content_20000_30000.csv', columns=['Push_Count', 'Content'])

# 爬30000到40000

In [85]:
link_all = all_res["Link"]
link_30000_40000 = link_all[30000:40000]
len(link_30000_40000)
link_30000_40000.head()
link_30000_40000.tail()

39995    /bbs/Gossiping/M.1555692377.A.55A.html
39996    /bbs/Gossiping/M.1555692399.A.BC0.html
39997    /bbs/Gossiping/M.1555692419.A.903.html
39998    /bbs/Gossiping/M.1555692433.A.F8C.html
39999    /bbs/Gossiping/M.1555692472.A.12A.html
Name: Link, dtype: object

In [81]:
content_30000_40000 = pd.DataFrame(columns=['Push_Count','Content'])
count = 0

if __name__ == '__main__':
    for link in link_30000_40000:
        df = getArticles(link)
        content_30000_40000 = pd.concat([content_30000_40000, df], ignore_index=True, sort=False)
        count += 1
        
        if count % 100 == 0:
            sleep(5)
            
content_30000_40000.head()

,Push_Count,Content
0,19,美國《神經學》期刊研究：腹部脂肪多 可能讓大腦萎縮\n\n\n2019-04-14 09:2...
1,8,肥宅剛剛查了一下，中國民主派人士，稱中國五毛為「小粉紅」。\n\n結果，看到來源，居然超級色...
2,8,1.媒體來源:\n中央社\n\n2.記者署名\n譯者：嚴思祺/核稿：劉學源\n\n3.完整新...
3,10,如題\n本人預計在今年暑假要跟未來老婆\n一起去義大利自助旅行\n\n耳聞義大利治安不好，宵...
4,7,: https://i.imgur.com/P031aDd.jpg\n: 中共軍機保衛我國空...


In [83]:
content_30000_40000.to_csv('content_30000_40000.csv', columns=['Push_Count', 'Content'])

# 爬40000到50000

In [84]:
link_all = all_res["Link"]
link_40000_50000 = link_all[40000:50000]
len(link_40000_50000)
link_40000_50000.head()

40000    /bbs/Gossiping/M.1555692476.A.589.html
40001    /bbs/Gossiping/M.1555692496.A.F14.html
40002    /bbs/Gossiping/M.1555692512.A.36C.html
40003    /bbs/Gossiping/M.1555692533.A.76E.html
40004    /bbs/Gossiping/M.1555692572.A.AA3.html
Name: Link, dtype: object

In [89]:
content_40000_50000 = pd.DataFrame(columns=['Push_Count','Content'])
count = 0

if __name__ == '__main__':
    for link in link_40000_50000:
        df = getArticles(link)
        content_40000_50000 = pd.concat([content_40000_50000, df], ignore_index=True, sort=False)
        count += 1
        
        if count % 100 == 0:
            sleep(5)
            
content_40000_50000.head()

,Push_Count,Content
0,9,胖子缺點很多\n\n會被歧視、排擠\n\n體型胖又擋路\n\n要維持身材就要一直運動\n\n...
1,7,有公然承認自己的經費就是請小邊(ㄨㄤˇㄐㄩㄣ)\n然後又直接想收編四個板主的\n然後又想把八...
2,1,安 你各位\n\n94啊，八卦板龜第一條有個內容是醬：\n\n ‧ 每日發文(含回文)額度，...
3,36,: ※ 引述《LukeSkywaker (路克天醒者)》之銘言：\n: : 我比較好奇Gov...
4,9,雖然不知道什麼時候才會開放申請新帳號，\n但是總是要做個準備！！！\n不可能ㄧ輩子都不開放新...


In [90]:
content_40000_50000.to_csv('content_40000_50000.csv', columns=['Push_Count', 'Content'])

# 爬50000到63300

In [95]:
link_all = all_res["Link"]
link_50000_63300 = link_all[50000:63301]
len(link_50000_63300)
link_50000_63300.head()
len(link_50000_63300)

13301

In [97]:
content_50000_63300 = pd.DataFrame(columns=['Push_Count','Content'])
count = 0

if __name__ == '__main__':
    for link in link_50000_63300:
        df = getArticles(link)
        content_50000_63300 = pd.concat([content_50000_63300, df], ignore_index=True, sort=False)
        count += 1
        
        if count % 100 == 0:
            sleep(5)
            
content_50000_63300.head()

,Push_Count,Content
0,3,大家別吵了\n其實我覺得喔\n大家說的\n都有道理\n像當老闆的\n一定要996才能成功\n...
1,6,不贊成\n\n只當一年兵要幹嘛？\n服役要完整規劃，從高中畢業就應該先入伍訓練一個暑假，開學...
2,40,客家菜就是要去客家縣市讓客家人做才對味。\n\n但是客家小炒點這個就low，一堆熱炒店都有了...
3,13,選手自己賭外圍\n\n打假賽果然是台灣賽事的傳統\n\n不過已經快兩年沒看LOL比賽了\n\...
4,49,是否有專板本板並非萬能問板\n兩則自刪及被刪也算兩篇之內\n\n本看板嚴格禁止政治問卦發文問...


In [99]:
content_50000_63300.to_csv('content_50000_63300.csv', columns=['Push_Count', 'Content'])

# 將6萬筆內文合併

In [103]:
content_all = pd.DataFrame()
content_all = pd.concat([content_0_10000 , content_10000_20000 , content_20000_30000 , content_30000_40000 , content_40000_50000 , content_50000_63300] , axis = 0 , ignore_index = True)
content_all

,Push_Count,Content
0,17,各位八卦鄉民們晚安\n\n年號通常有皇帝對於未來期許的意味\n\n如果八卦板也來年號紀元\n...
1,27,1.媒體來源:\n\n\n自由\n2.完整新聞標題:\n\n高雄簽13億無法律效力MOU 陳...
2,27,台灣人最愛騎機車，\n\n一遇到下雨就只能穿雨衣，\n\n但雨衣的設計根本就沒有辦法騎機車，...
3,10,大紀元\n\n\n紐航將開通直飛首爾航班 機上Wifi將免費\n\n\n【大紀元2019年0...
4,8,之前有個自稱聾啞的外國人拿著國旗給路人\n路人如果伸手拿了 就會拿出張紙寫一支國旗100塊...
5,101,自由\n\n去中國工作待遇好？蔡依橙爆上海、台北起薪差很大\n\n〔財經頻道／綜合報導〕近年...
6,10,晚上好\n相信大家多少都知道一點人類起源跟各種宇宙誕生的東西\n\n我蠻好奇的是\n為啥一直...
7,8,八卦版上 有不少人喜歡反串\n然後再換個帳號嗆反串 也達成洗文章聲量\n然後就延伸 反反串 ...
8,51,4說前陣子我大 屁弟派 終於輸給 印度阿三惹\n\n後來查惹查一下世界訂閱數的排名\n\n清...
9,48,臉書卦點說明：（繁體中文 20 個字）\n知名樂團拷秋勤針對柯屁和韓導兩人閃躲共軍越過海峽中...


In [116]:
content_all.to_csv('content_all.csv', columns=['Push_Count', 'Content'])

In [113]:
#測試內容與標題有無錯誤

message_all = content_all["Content"]
m = pd.Series(message_all[33209])
t = pd.Series(all_res_title[33209])
df = pd.DataFrame()
df["Title"] = t
df["Content"] = m
df

,Title,Content
0,[問卦] 移民的目的國家推薦？？,感覺戰爭的腳步越來越近了\n\n想說是不是該來找一個地方移民，當作以後的退路\n\n具體條件...


# 將文章列表與內文合併

In [164]:
all_res.head()

,Unnamed: 0,Date,Push,Title,Author,Link
0,0,4/01,NaN,[問卦] 八卦板取年號該取啥,med5566,/bbs/Gossiping/M.1554113138.A.5BC.html
1,1,4/01,10,[新聞] 高雄簽13億無法律效力MOU 陳吉仲：農委,qqq87112,/bbs/Gossiping/M.1554113163.A.4CE.html
2,2,4/01,5,[問卦] 有沒有雨衣只是把雨水集中到鞋子裡的八卦,ken0062,/bbs/Gossiping/M.1554113195.A.80D.html
3,3,4/01,5,[新聞] 紐航將開通直飛首爾航班 機上Wifi將免費,FreeWiFi,/bbs/Gossiping/M.1554113228.A.0A0.html
4,4,4/01,NaN,[問卦] 有沒有賣國旗的外國人的八卦,WWTDD,/bbs/Gossiping/M.1554113273.A.A5E.html


In [165]:
content_all.head()

,Push_Count,Content
0,17,各位八卦鄉民們晚安\n\n年號通常有皇帝對於未來期許的意味\n\n如果八卦板也來年號紀元\n...
1,27,1.媒體來源:\n\n\n自由\n2.完整新聞標題:\n\n高雄簽13億無法律效力MOU 陳...
2,27,台灣人最愛騎機車，\n\n一遇到下雨就只能穿雨衣，\n\n但雨衣的設計根本就沒有辦法騎機車，...
3,10,大紀元\n\n\n紐航將開通直飛首爾航班 機上Wifi將免費\n\n\n【大紀元2019年0...
4,8,之前有個自稱聾啞的外國人拿著國旗給路人\n路人如果伸手拿了 就會拿出張紙寫一支國旗100塊...


In [166]:
df = pd.concat([all_res , content_all] , axis = 1 , ignore_index = True)
df_0 = df.rename(index = str , columns = { 0:"non" , 1:"Date", 2:"Push" , 3:"Title" , 4:"Author" , 5:"Link" , 6:"Push_Count" , 7:"Content" })
April_Gossiping_Data = df_0.drop(columns = ["non"])

t = df_1["Title"]
c = df_1["Content"]
#print(t[33333])
#print(c[33333])

April_Gossiping_Data.head()

,Date,Push,Title,Author,Link,Push_Count,Content
0,4/01,NaN,[問卦] 八卦板取年號該取啥,med5566,/bbs/Gossiping/M.1554113138.A.5BC.html,17,各位八卦鄉民們晚安\n\n年號通常有皇帝對於未來期許的意味\n\n如果八卦板也來年號紀元\n...
1,4/01,10,[新聞] 高雄簽13億無法律效力MOU 陳吉仲：農委,qqq87112,/bbs/Gossiping/M.1554113163.A.4CE.html,27,1.媒體來源:\n\n\n自由\n2.完整新聞標題:\n\n高雄簽13億無法律效力MOU 陳...
2,4/01,5,[問卦] 有沒有雨衣只是把雨水集中到鞋子裡的八卦,ken0062,/bbs/Gossiping/M.1554113195.A.80D.html,27,台灣人最愛騎機車，\n\n一遇到下雨就只能穿雨衣，\n\n但雨衣的設計根本就沒有辦法騎機車，...
3,4/01,5,[新聞] 紐航將開通直飛首爾航班 機上Wifi將免費,FreeWiFi,/bbs/Gossiping/M.1554113228.A.0A0.html,10,大紀元\n\n\n紐航將開通直飛首爾航班 機上Wifi將免費\n\n\n【大紀元2019年0...
4,4/01,NaN,[問卦] 有沒有賣國旗的外國人的八卦,WWTDD,/bbs/Gossiping/M.1554113273.A.A5E.html,8,之前有個自稱聾啞的外國人拿著國旗給路人\n路人如果伸手拿了 就會拿出張紙寫一支國旗100塊...


In [167]:
April_Gossiping_Data.to_csv('April_Gossiping_Data.csv', columns=['Date', 'Push','Title','Author','Link','Push_Count','Content'])

In [174]:
pd.read_csv("April_Gossiping_Data.csv")

,Unnamed: 0,Date,Push,Title,Author,Link,Push_Count,Content
0,0,4/01,NaN,[問卦] 八卦板取年號該取啥,med5566,/bbs/Gossiping/M.1554113138.A.5BC.html,17,各位八卦鄉民們晚安\n\n年號通常有皇帝對於未來期許的意味\n\n如果八卦板也來年號紀元\n...
1,1,4/01,10,[新聞] 高雄簽13億無法律效力MOU 陳吉仲：農委,qqq87112,/bbs/Gossiping/M.1554113163.A.4CE.html,27,1.媒體來源:\n\n\n自由\n2.完整新聞標題:\n\n高雄簽13億無法律效力MOU 陳...
2,2,4/01,5,[問卦] 有沒有雨衣只是把雨水集中到鞋子裡的八卦,ken0062,/bbs/Gossiping/M.1554113195.A.80D.html,27,台灣人最愛騎機車，\n\n一遇到下雨就只能穿雨衣，\n\n但雨衣的設計根本就沒有辦法騎機車，...
3,3,4/01,5,[新聞] 紐航將開通直飛首爾航班 機上Wifi將免費,FreeWiFi,/bbs/Gossiping/M.1554113228.A.0A0.html,10,大紀元\n\n\n紐航將開通直飛首爾航班 機上Wifi將免費\n\n\n【大紀元2019年0...
4,4,4/01,NaN,[問卦] 有沒有賣國旗的外國人的八卦,WWTDD,/bbs/Gossiping/M.1554113273.A.A5E.html,8,之前有個自稱聾啞的外國人拿著國旗給路人\n路人如果伸手拿了 就會拿出張紙寫一支國旗100塊...
5,5,4/01,25,[新聞] 去中國工作待遇好？蔡依橙爆上海、台北起,visualsense,/bbs/Gossiping/M.1554113286.A.B4F.html,101,自由\n\n去中國工作待遇好？蔡依橙爆上海、台北起薪差很大\n\n〔財經頻道／綜合報導〕近年...
6,6,4/01,4,[問卦] 所以為什麼要隱瞞有蜥蜴人?,Secret69,/bbs/Gossiping/M.1554113303.A.906.html,10,晚上好\n相信大家多少都知道一點人類起源跟各種宇宙誕生的東西\n\n我蠻好奇的是\n為啥一直...
7,7,4/01,4,[問卦] 平常如果都反串 愚人節時該怎麼辦?,kaimu457,/bbs/Gossiping/M.1554113343.A.287.html,8,八卦版上 有不少人喜歡反串\n然後再換個帳號嗆反串 也達成洗文章聲量\n然後就延伸 反反串 ...
8,8,4/01,26,[問卦] 為什麼華人YT訂閱數難以超越外國人?,royroy666,/bbs/Gossiping/M.1554113378.A.960.html,51,4說前陣子我大 屁弟派 終於輸給 印度阿三惹\n\n後來查惹查一下世界訂閱數的排名\n\n清...
9,9,4/01,X1,[臉書] 拷秋勤 Kou Chou Ching,cycling,/bbs/Gossiping/M.1554113398.A.193.html,48,臉書卦點說明：（繁體中文 20 個字）\n知名樂團拷秋勤針對柯屁和韓導兩人閃躲共軍越過海峽中...
